In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('./data/train.csv',index_col='index')
train.head()

/home/frodos/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,event_id,X,Y,Z,TX,TY,chi2,signal,brick_number
index,,,,,,,,,
0,-999,66162.843750,65620.031250,0.0,0.166890,0.276674,2.988714,0.0,44
1,-999,43900.445312,67091.671875,0.0,-0.179967,-0.478795,1.901420,0.0,98
2,-999,37564.019531,26618.890625,55599.0,-0.267820,-0.282087,2.510245,0.0,64
3,-999,39380.027344,48135.425781,19395.0,0.265197,-0.577493,1.215219,0.0,63
4,-999,65040.781250,44018.468750,12930.0,-0.711161,-0.245592,2.973661,0.0,2


In [3]:
# оценим количество кирпичей
train.brick_number.unique().shape

(100,)

In [4]:
brick2 = train.query('brick_number == 2')

In [5]:
brick2.query('signal == 1').event_id.unique()

array([15847, 20217,  4299])

Примерный план обработки:
    <ul>
    <li> Разобрать треки по кирпичам
    <li> Отсортировать по расстоянию z, т.е. по слоям. Подумать о введении дополнительной переменной "слой"
    <li> Отсортировать по координате X
    <li> Начать перебор с начала кирпича.
    </ul>

In [6]:
brick = brick2

In [7]:
# create dictionaries
zPos = np.sort(brick.Z.unique())
zPosDict = dict(enumerate(zPos)) # dictionary of positions (layerNumber => position)
zDiffDict = dict(enumerate(np.diff(zPos))) # dictionaru of distances (layerNumber => distance to the next layer)
layerDict = {v: k for k, v in zPosDict.iteritems() } # dictionary of layer number (position => layerNumber)

# add extra feature - layer number
brick = brick.assign(layerId = brick.Z.map(layerDict))

In [125]:
idx1 = 3901599
idx2 = 3670692
layerId = brick.loc[idx1].layerId
prediction = getPosition(brick.loc[idx1],zDiffDict.get(layerId))
prediction

array([  3.54064776e+04,   2.88788655e+04,   9.66750383e-02,
         5.11160716e-02])

In [38]:
layerId = 1
layer0 = brick.query('layerId == @layerId')
layer1 = brick.query('layerId == @layerId+1').sort_values(by='X',ascending=True)

In [39]:
layer0Arr = layer0.reset_index().as_matrix(columns = ['index', 'X', 'Y', 'TX', 'TY', 'signal'])
layer1Arr = layer1.reset_index().as_matrix(columns = ['index', 'X', 'Y', 'TX', 'TY', 'signal'])

In [43]:
# перепишу по массивы
#%%time
result  = pd.DataFrame(columns=layer1.columns.names)
dx  = 100 # estimation of error along OX axis
dy  = 100 # estimation of error along OY axis
dtx = 0.2
dty = 0.2

dz = zDiffDict.get(layerId)
for track in layer0Arr:
    #calculate new pos
    newX  = track[1]+dz*np.tan(track[3])
    newY  = track[2]+dz*np.tan(track[4])
    newTX = track[3]
    newTY = track[4]
    
    #print 
    lEdge = np.searchsorted(layer1Arr[:,1],newX-dx,side='left')
    rEdge = np.searchsorted(layer1Arr[:,1],newX+dx,side='right')
    subSet = layer1Arr[lEdge:rEdge,:]
    subSet = subSet[np.where((subSet[:,2]>newY-dy) &  (subSet[:,2]<newY+dy) & (subSet[:,3]>newTX-dtx) & (subSet[:,3]<newTX+dtx) & (subSet[:,4]>newTY-dty) & (subSet[:,4]<newTX+dty)),:][0]
    
    if subSet.shape[0] > 0:
        print subSet
        print '***'

[[  3.47483000e+05   6.21990547e+04   7.32938281e+03   1.78683043e-01
    4.38244045e-02   0.00000000e+00]]
***
[[  1.10442200e+06   5.12175742e+04   7.55027422e+04   7.36607146e-03
    2.69456834e-01   0.00000000e+00]]
***


In [31]:
subSet

array([], shape=(1, 0, 6), dtype=float64)